### MACCROBAT_DATA

Annotation files are in brat standoff format

General annotation structure
All annotations follow the same basic structure: Each line contains one annotation, and each annotation is given an ID that appears first on the line, separated from the rest of the annotation by a single TAB character. The rest of the structure varies by annotation type.

Examples of annotation for an entity (T1), an event trigger (T2), an event (E1) and a relation (R1) are shown in the following.

Annotation ID conventions
All annotations IDs consist of a single upper-case character identifying the annotation type and a number. The initial ID characters relate to annotation types as follows:

T: text-bound annotation
R: relation
E: event
A: attribute
M: modification (alias for attribute, for backward compatibility)
N: normalization [new in v1.3]
#: note

In [33]:
MACCROBAT_data_dir = "./data/MACCROBAT"

In [41]:
import os
import pandas as pd

In [3]:
file_ids = []
for file in os.listdir(MACCROBAT_data_dir):
    file_id = file.split(".")[0]
    if file_id not in file_ids:
        file_ids.append(file_id)
file_ids

['19860925',
 '26361640',
 '26228535',
 '27773410',
 '23678274',
 '25853982',
 '28103924',
 '27064109',
 '28154700',
 '20146086',
 '26656340',
 '28353558',
 '22515939',
 '28353588',
 '26309459',
 '28272235',
 '23242090',
 '23312850',
 '23124805',
 '26106249',
 '26313770',
 '26285706',
 '18416479',
 '28353613',
 '28151916',
 '26175648',
 '23468586',
 '28216610',
 '27059701',
 '28121940',
 '23077697',
 '27741115',
 '21067996',
 '28100235',
 '28151860',
 '25884600',
 '27904130',
 '19214295',
 '18787726',
 '22719160',
 '28422883',
 '26675562',
 '21477357',
 '25139918',
 '28353561',
 '22791498',
 '28538413',
 '26457578',
 '27842605',
 '20671919',
 '25155594',
 '26469535',
 '28353604',
 '28403092',
 '28239141',
 '28202869',
 '25024632',
 '28403086',
 '18666334',
 '25572898',
 '28296775',
 '22514576',
 '26584481',
 '28296749',
 '16778410',
 '19860007',
 '28190872',
 '25743872',
 '26523273',
 '28193213',
 '28120581',
 '26670309',
 '26336183',
 '25410883',
 '26530965',
 '28057913',
 '20977862',

In [32]:
tags = []
for ann_file in [os.path.join(MACCROBAT_data_dir, file_id+".ann") for file_id in file_ids]:
    with open(ann_file) as f:
        for line in f.readlines():
            tags.append(line.split("\t")[0][0])

set(tags)

{'#', '*', 'A', 'E', 'R', 'T'}

For named entity recognition, only the T tags are needed to generate the BIO format. The other tags such as #, *, A, E, and R are used for representing different types of information in the .ann files and are not directly related to named entity recognition.

\# is used to denote comments in the annotation file.
\* is used to represent coreference annotations.
A is used to represent attribute annotations.
E is used to represent event annotations.
R is used to represent relation annotations.

Therefore, we can ignore all other tags except for the T tags when generating the BIO format for named entity recognition.

### Find Entity types in ann files

In [177]:
# create an empty list to store the DataFrames
df_list = []

for ann_file in [os.path.join(MACCROBAT_data_dir, file_id+".ann") for file_id in file_ids]:
    # read the TSV file with consecutive tabs treated as a single delimiter
    df = pd.read_csv(ann_file, sep="\t+", header=None, names=['id', 'entity_with_range', 'word'], engine='python')

    # selecting only T tag rows
    df = df[df.iloc[:,0].str.startswith('T')]

    df['file_name'] = f"{os.path.splitext(ann_file)[0]}.txt"

    df_list.append(df)

ann_df = pd.concat(df_list)
ann_df = ann_df.reset_index(drop=True)

In [178]:
ann_df.head()

,id,entity_with_range,word,file_name
0,T1,Age 4 15,24-year-old,./data/MACCROBAT/19860925.txt
1,T2,Sex 28 32,male,./data/MACCROBAT/19860925.txt
2,T3,History 16 27,non-smoking,./data/MACCROBAT/19860925.txt
3,T4,Clinical_event 41 50,presented,./data/MACCROBAT/19860925.txt
4,T5,Sign_symptom 65 75,hemoptysis,./data/MACCROBAT/19860925.txt


In [179]:
ann_df.shape

(25041, 4)

In [180]:
ann_df[ann_df['entity_with_range'].str.split().str.len() > 3]

,id,entity_with_range,word,file_name
39,T40,Disease_disorder 701 714;730 735,granular cell tumor,./data/MACCROBAT/19860925.txt
1190,T31,Dosage 4085 4093;4103 4108,low dose daily,./data/MACCROBAT/20146086.txt
1272,T160,Dosage 3117 3124;3135 3141,1500 mg weekly,./data/MACCROBAT/20146086.txt
1469,T101,Detailed_description 2552 2559;2585 2654,neither had developed any signs or symptoms su...,./data/MACCROBAT/28353558.txt
2376,T42,Administration 704 715;721 726,intravenous bolus,./data/MACCROBAT/23124805.txt
2380,T47,Dosage 828 835;861 879,5 mg/kg every 4 to 6 weeks,./data/MACCROBAT/23124805.txt
2381,T48,Dosage 849 857;861 879,10 mg/kg every 4 to 6 weeks,./data/MACCROBAT/23124805.txt
2853,T18,Diagnostic_procedure 341 350;357 358,Hepatitis C,./data/MACCROBAT/18416479.txt
3335,T16,Disease_disorder 315 317;319 330,LV dysfunction,./data/MACCROBAT/23468586.txt
3336,T15,Disease_disorder 297 313;319 330,left ventricular dysfunction,./data/MACCROBAT/23468586.txt


The above entries represents the tagged text spans more than one disjoint ranges

Note that the "B" prefix is used to indicate the beginning of an entity, while the "I" prefix is used to indicate an intermediate token within an entity. When there are disjoint ranges for an entity, we can start a new entity with a "B" prefix for each range.

Including the full text that spans disjoint ranges in the BIO format will help the model learn to recognize the entire entity, even if it is fragmented across multiple parts of the text.

In [181]:
ann_df['entity'] = ann_df['entity_with_range'].str.split().str[0]
ann_df['range'] = ann_df['entity_with_range'].str.split().str[1:]
ann_df = ann_df[['id', 'entity', 'range', 'word', 'file_name']]
ann_df.head()

,id,entity,range,word,file_name
0,T1,Age,"[4, 15]",24-year-old,./data/MACCROBAT/19860925.txt
1,T2,Sex,"[28, 32]",male,./data/MACCROBAT/19860925.txt
2,T3,History,"[16, 27]",non-smoking,./data/MACCROBAT/19860925.txt
3,T4,Clinical_event,"[41, 50]",presented,./data/MACCROBAT/19860925.txt
4,T5,Sign_symptom,"[65, 75]",hemoptysis,./data/MACCROBAT/19860925.txt


In [192]:
print("The set of entities available in the dataset is as follows: ")
set(ann_df['entity'])

The set of entities available in the dataset is as follows: 


{'Activity',
 'Administration',
 'Age',
 'Area',
 'Biological_attribute',
 'Biological_structure',
 'Clinical_event',
 'Color',
 'Coreference',
 'Date',
 'Detailed_description',
 'Diagnostic_procedure',
 'Disease_disorder',
 'Distance',
 'Dosage',
 'Duration',
 'Family_history',
 'Frequency',
 'Height',
 'History',
 'Lab_value',
 'Mass',
 'Medication',
 'Nonbiological_location',
 'Occupation',
 'Other_entity',
 'Other_event',
 'Outcome',
 'Personal_background',
 'Qualitative_concept',
 'Quantitative_concept',
 'Severity',
 'Sex',
 'Shape',
 'Sign_symptom',
 'Subject',
 'Texture',
 'Therapeutic_procedure',
 'Time',
 'Volume',
 'Weight'}

### Exploring each entities

In [314]:
ann_df.groupby(['entity']).size().sort_values(ascending=False)

entity
Diagnostic_procedure      4567
Sign_symptom              3359
Biological_structure      2931
Detailed_description      2901
Lab_value                 2858
Disease_disorder          1362
Medication                1076
Therapeutic_procedure     1005
Date                       731
Clinical_event             626
History                    392
Severity                   369
Dosage                     362
Nonbiological_location     354
Coreference                313
Duration                   280
Age                        206
Sex                        191
Administration             175
Distance                   122
Activity                   108
Family_history              81
Frequency                   76
Shape                       65
Time                        57
Personal_background         57
Subject                     54
Color                       52
Texture                     46
Area                        43
Outcome                     42
Qualitative_concept         41
V

In [222]:
# Define a function to select random rows from each group
def select_random_rows(group):
    return group.sample(n=min(2, len(group)), replace=False)

# Apply the function to the DataFrame grouped by 'group'
random_rows = ann_df.groupby('entity').apply(select_random_rows)

# Print the selected random rows
random_rows['word']

entity               
Activity        16278               flexed
                7071     physical exercise
Administration  16777                 oral
                11207              topical
Age             19252          23-year-old
                               ...        
Time            6758        first 24 hours
Volume          18706        11 × 8 × 2 cm
                24524                 5 mL
Weight          24555               3132 g
                5538                 83 kg
Name: word, Length: 82, dtype: object

In [223]:
selected_entities = ['Age', 'Biological_attribute', 'Biological_structure', 'Clinical_event', 'Diagnostic_procedure', 'Disease_disorder', 'Dosage', 'Family_history', 'Height', 'History', 'Lab_value', 'Mass', 'Medication', 'Sex', 'Sign_symptom', 'Therapeutic_procedure', 'Weight']

In [227]:
filtered_ann_df = ann_df[ann_df['entity'].isin(selected_entities)]
filtered_ann_df = filtered_ann_df.reset_index(drop=True)
filtered_ann_df.head()

,id,entity,range,word,file_name
0,T1,Age,"[4, 15]",24-year-old,./data/MACCROBAT/19860925.txt
1,T2,Sex,"[28, 32]",male,./data/MACCROBAT/19860925.txt
2,T3,History,"[16, 27]",non-smoking,./data/MACCROBAT/19860925.txt
3,T4,Clinical_event,"[41, 50]",presented,./data/MACCROBAT/19860925.txt
4,T5,Sign_symptom,"[65, 75]",hemoptysis,./data/MACCROBAT/19860925.txt


In [228]:
filtered_ann_df.shape

(19036, 5)

## Generating BIO format files

In [ ]:
import re

#### Function to read txt files

In [238]:
import numpy as np
import string

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

In [232]:
def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    return content

In [281]:
import re

# Read the txt file
with open('./data/MACCROBAT/19860925.txt', 'r') as f:
    text = f.read()

# Read the ann file
with open('./data/MACCROBAT/19860925.ann', 'r') as f:
    ann = f.readlines()

# Define a function to convert the ann file into a list of (label, start, end) tuples
def parse_ann(ann):
    entities = []
    for line in ann:
        fields = line.strip().split('\t')
        if fields[0].startswith('T'):
            entity_with_range, word = fields[1], fields[2]
            label = entity_with_range.split()[0]
            if label in selected_entities:
                ranges = [
                    (
                        int(start_end.split()[0]),
                        int(start_end.split()[1])
                    )
                    for start_end in ' '.join(entity_with_range.split()[1:]).split(';')
                ]
                for start, end in ranges:
                    entities.append((label, start, end))
    return entities

# Convert the ann file into a list of (label, start, end) tuples
entities = parse_ann(ann)

# Define a function to convert the entities into a list of (token, tag) tuples in BIO format
def entities_to_bio(text, entities):
    tokens = re.findall(r'\b\w+\b', text)
    tags = ['O'] * len(tokens)
    for label, start, end in entities:
        for i in range(len(tokens)):
            token_start = text.find(tokens[i])
            if start <= token_start < end:
                if token_start == start:
                    tags[i] = 'B-' + label
                else:
                    tags[i] = 'I-' + label
    return list(zip(tokens, tags))



# Convert the entities into a list of (token, tag) tuples in BIO format
bio = entities_to_bio(text, entities)


In [284]:
# Print the result
for word, tag in bio:
    print(word, tag)

Our O
24 B-Age
year I-Age
old I-Age
non B-History
smoking I-History
male B-Sex
patient O
presented B-Clinical_event
with O
repeated O
hemoptysis B-Sign_symptom
in I-History
May O
2008 O
with O
4 I-Age
days O
of O
concomitant O
right B-Biological_structure
thoracic I-Biological_structure
pain B-Sign_symptom
which O
intensified O
while O
breathing O
During O
holidays B-History
in I-History
his I-History
home I-History
country I-History
this O
Cuban O
patient O
suffered O
from O
a I-Age
cold B-Sign_symptom
with O
fever B-Sign_symptom
and O
a I-Age
strong O
cough B-Sign_symptom
The O
strong O
dry O
cough B-Sign_symptom
persisted O
after O
recovery O
from O
the O
cold B-Sign_symptom
The O
patient O
did O
not O
report O
any O
loss B-Sign_symptom
of O
weight I-Sign_symptom
The O
initial O
CT B-Diagnostic_procedure
scan I-Diagnostic_procedure
of O
the O
thorax B-Biological_structure
showed O
a I-Age
12 O
4 I-Age
cm O
solid O
mass B-Sign_symptom
paravertebral B-Biological_structure
right B-Biol

In [285]:
MACCROBAT_data_dir

'./data/MACCROBAT'

In [311]:
def convert_ann_to_bio(input_dir, output_dir):
    for file in os.listdir(input_dir):
        if file.endswith('.txt'):
            # Read the corresponding txt file
            with open(os.path.join(input_dir, file), 'r', encoding='utf-8') as f:
                text = f.read()
            # Find the corresponding ann file
            ann_file = os.path.join(input_dir, file.replace('.txt', '.ann'))

            # tokenization
            tokens = re.findall(r'\b\w+\b', text)
            # Initialize a list to hold the BIO-formatted tags
            bio_tags = ['O'] * len(tokens)

            # Read the annotation file
            with open(ann_file, 'r', encoding='utf-8') as f:
                for line in f:
                    fields = line.strip().split('\t')
                    # Get the tag and its starting and ending positions
                    if fields[0].startswith('T'):
                        entity_with_range, word = fields[1], fields[2]
                        label = entity_with_range.split()[0]
                        if label in selected_entities:
                            ranges = [
                                (
                                    int(start_end.split()[0]),
                                    int(start_end.split()[1])
                                )
                                for start_end in ' '.join(entity_with_range.split()[1:]).split(';')
                            ]
                            # Keep track of the current position in the text
                            current_pos = 0
                            for i in range(len(tokens)):
                                # Calculate the starting position of the token
                                token_start = text.find(tokens[i], current_pos)
                                token_end = token_start + len(tokens[i])
                                # Update the current position in the text
                                current_pos = token_end
                                for start, end in ranges:
                                    if start <= token_start and end >= token_end:
                                        if token_start == start:
                                            bio_tags[i] = 'B-' + label
                                        else:
                                            bio_tags[i] = 'I-' + label
            # for token, tag in zip(tokens, bio_tags):
            #     print(token, tag)


            if not os.path.exists(output_dir):
                os.makedirs(output_dir)
             # # Write the BIO tags to a new file
            with open(os.path.join(output_dir, file.replace('.txt', '.bio')), 'w', encoding='utf-8') as f:
                for i in range(len(tokens)):
                    f.write(tokens[i] + '\t' + bio_tags[i] + '\n')
                f.write('\n')

In [312]:
convert_ann_to_bio('./data/MACCROBAT', './data/BIO_FILES')